In [ ]:
import os
import json
import torch
import numpy as np
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset

: 

In [1]:
# Define the base directory where your protein folders are located
print(os.getcwd())
base_dir = '/path/to/your/protein/folders'

# List to store loaded proteins
all_proteins = []

# Iterate through the folders 'cv0' to 'cv5'
for i in range(6):  # Assuming folders are named 'cv0' to 'cv5'
    folder_name = f'cv{i}'
    folder_path = os.path.join(base_dir, folder_name)

    # Iterate through files in the folder
    for filename in os.listdir(folder_path):
        if filename.endswith('.npy'):
            file_path = os.path.join(folder_path, filename)

            # Load the protein and append to the list
            protein = np.load(file_path, allow_pickle=True).item()
            all_proteins.append(protein)

# Now, all_proteins contains the loaded proteins from all folders


NameError: name 'os' is not defined

In [ ]:
# Assuming X and y are your feature matrix and target variable, as in the scikit-learn example
print(os.getcwd())
f = open("DeepTMHMM.partitions.json")

labels = json.load(f)
print(labels.keys())
encoder_path = f"../encoder_proteins/{cv}/{protein['id']}"
read_dictionary = np.load(encoder_path + ".npy", allow_pickle='TRUE').item()
print(read_dictionary)
# Convert data to PyTorch tensors
X = torch.tensor(X.values, dtype=torch.float32)
y = torch.tensor(y.values, dtype=torch.float32)

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Create DataLoader for training and testing sets
train_dataset = TensorDataset(X_train, y_train)
test_dataset = TensorDataset(X_test, y_test)

train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)

In [ ]:
class DenseNN(nn.Module):
    def __init__(self, input_size, num_classes):
        super(DenseNN, self).__init__()
        
        # Flatten the input to a 1D array
        self.flatten = nn.Flatten()

        # Define fully connected layers
        self.fc1 = nn.Linear(input_size, 256)
        self.relu1 = nn.ReLU()
        self.fc2 = nn.Linear(256, 128)
        self.relu2 = nn.ReLU()

        # Output layer with softmax activation for multiclass classification
        self.fc3 = nn.Linear(128, num_classes)
        self.softmax = nn.Softmax(dim=1)

    def forward(self, x):
        x = self.flatten(x)
        x = self.relu1(self.fc1(x))
        x = self.relu2(self.fc2(x))
        x = self.softmax(self.fc3(x))
        return x

# Specify the input size (L * 512) and number of classes (6)
input_size = L * 512
num_classes = 6

# Create the model
dense_nn_model = DenseNN(input_size, num_classes)

# Print the model architecture
print(dense_nn_model)


In [ ]:
model = dense_nn_model(input_size, num_classes)
#criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.01)

# Training loop
num_epochs = 50
for epoch in range(num_epochs):
    for inputs, labels in train_loader:
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels.view(-1, 1))
        loss.backward()
        optimizer.step()

# Evaluation on the test set
model.eval()
with torch.no_grad():
    all_predictions = []
    all_labels = []
    for inputs, labels in test_loader:
        outputs = model(inputs)
        predictions = (outputs > 0.5).float()
        all_predictions.extend(predictions.numpy())
        all_labels.extend(labels.numpy())

# Calculate accuracy
accuracy = accuracy_score(all_labels, all_predictions)
print(f"Accuracy: {accuracy}")
